# Eval IoU of Segmentation model with different inpul image angle for the rotation equivariance model

In [1]:
%matplotlib inline
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import models
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import torch.nn.functional as F
import torchvision.utils as vutils
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.dataset import random_split
import numpy as np
import matplotlib.pyplot as plt
import pickle
import ignite.distributed as idist
from ignite.contrib.engines import common
from ignite.contrib.handlers import ProgressBar
from ignite.engine import Engine, Events, create_supervised_evaluator
from ignite.metrics import Accuracy
from ignite.metrics import ConfusionMatrix, mIoU,Accuracy,Loss
sys.path.insert(1, '../../utils')
sys.path.insert(1, '../../datasets')
sys.path.insert(1, '../../models')
import utils as U
from deeplab import Res_Deeplab
from utils import * 
import eval_train as ev
import my_datasets as mdset
#from fcn8 import FCN8_VGG16

In [2]:
SAVE_DIR = '~/data/model'
dataroot_voc = '~/data/voc2012'
dataroot_sbd = '~/data/sbd'


In [3]:
batch_size = 4
rotate = False
criterion_unsupervised = nn.CrossEntropyLoss(ignore_index=21)
#criterion_supervised = nn.CrossEntropyLoss(ignore_index=21) # On ignore la classe border.
Loss = 'CE' # Loss = 'KL' or 'CE' or None for L1,MSE…
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device :",device)

device : cuda:0


In [4]:
#train_dataset_VOC = mdset.VOCSegmentation(dataroot_voc,year='2012', image_set='train', download=True,rotate=rotate)
val_dataset_VOC = mdset.VOCSegmentation(dataroot_voc,year='2012', image_set='val', download=True,size_img=(321,321))
#train_dataset_SBD = mdset.SBDataset(dataroot_sbd, image_set='train_noval',mode='segmentation',rotate=rotate)

Using downloaded and verified file: /users/k/karmimy/data/voc2012/VOCtrainval_11-May-2012.tar
Using downloaded and verified file: /users/k/karmimy/data/voc2012/VOCtrainval_11-May-2012.tar


In [5]:
dataloader_train_VOC = torch.utils.data.DataLoader(train_dataset_VOC, batch_size=batch_size,\
                                                       shuffle=True,drop_last=True)
dataloader_val = torch.utils.data.DataLoader(val_dataset_VOC, batch_size=batch_size)

## Load model

In [6]:
def load_model(file=None,fcn=False):
    if file is None:
        if fcn is False:
            model = torchvision.models.segmentation.deeplabv3_resnet101(pretrained=True)
        else:
            model = torchvision.models.segmentation.fcn_resnet101(pretrained=True)
    else:
        model = torch.load(os.path.join(SAVE_DIR,file))
    return model

In [7]:
model = Res_Deeplab()
model.load_state_dict(torch.load('/share/homes/karmimy/equiv/save_model/adv_seg/resnet101COCO-41f33a49.pth'))
model.to(device)

track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
  )
  (layer2): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(256, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats

In [8]:
def eval_model(model,val_loader,device='cpu',num_classes=21):

    def evaluate_function(engine, batch):
        model.eval()
        with torch.no_grad():
            img, mask = batch
            img = img.to(device)
            mask = mask.to(device)
            mask_pred = model(img)
            mask_pred = F.interpolate(mask_pred,(520,520),mode='bilinear')
            try:
                mask_pred = mask_pred['out'] 
            except:
                pass
            return mask_pred, mask

    val_evaluator = Engine(evaluate_function)
    cm = ConfusionMatrix(num_classes=num_classes)
    mIoU(cm).attach(val_evaluator, 'mean IoU')   
    Accuracy().attach(val_evaluator, "accuracy")
    state = val_evaluator.run(val_loader)
    #print("mIoU :",state.metrics['mean IoU'])
    #print("Accuracy :",state.metrics['accuracy'])
    #print("CE Loss :",state.metrics['CE Loss'])
    
    return state

In [9]:
state = eval_model(model,dataloader_val,device=device)

In [10]:
state['mean IoU']

TypeError: 'State' object is not subscriptable

In [12]:
state.metrics['mean IoU']

0.03559876070028961

In [13]:
state

State:
	iteration: 363
	epoch: 1
	epoch_length: 363
	max_epochs: 1
	output: <class 'tuple'>
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: torch.utils.data.dataloader.DataLoader
	seed: <class 'NoneType'>
	times: <class 'dict'>